##### We are going to use real world demand data from one of our retail customers to show you how to query and integrate event features into a forecasting model to get a 20%+ RMSE improvement in forecasting accuracy.


Note: the purpose of this demo is to focus on how to use event features and integrate them into your existing set for your forecasting model, so I won't focus on how to tune, train or cross-validate an XGBoost model. Although I am using the Sagemaker computing platform and XGBoost model as an exmaple here, this demo architecutre is agnostic to computing platforms and forecasting algorithms.


In [ ]:
!pip --disable-pip-version-check install pandas
!pip --disable-pip-version-check install numpy
!pip --disable-pip-version-check install xgboost
!pip --disable-pip-version-check install predicthq

In [ ]:
from datetime import datetime, date, timedelta

import matplotlib.pyplot as plt
import pandas as pd
import plotly.graph_objects as go
import requests
from predicthq import Client
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor, plot_importance

### Model Architecture Design

![Picture title](data/workflow.png)

### 

### Background

We will predict the sales of energy drinks for a retail store located in Iowa City, Iowa. We have 1 year historical data, dated back from 2021-06-01 to 2022-07-04. We will use 2021-06-01 to 2022-06-19 as training data, and then predict the demand for the dates between 2022-06-20 to 2022-07-04.

We use a suggested radius of 1.76km to search for the events near this store. There are 23 venues close to this store within a 1.76km radius. Reach out to us if you would like us to determine a suggested radius for your locations.  

By running the Beam and Category Importance models, we learn there are eight event categories statistically correlated to the demand. These are:
- Sports
- Public holidays
- School holidays
- Expos
- Observances
- Severe weather
- Concerts
- Performing arts

Please get in touch to run your demand data through the Category Importance model, and get access to the Suggested Radius API.

### 

### Get relevant event features through Features API

In [ ]:
category_important_results = [
    "public_holidays",
    "sports",
    "school_holidays",
    "expos",
    "observances",
    "severe_weather",
    "concerts",
    "performing_arts",
]

The **ACCESS_TOKEN** is used for preparing event features from [Features API](https://docs.predicthq.com/start/features-api/). The provided ACCESS_TOKEN  is limited to the demo example. For event features in other locations or time periods, the following link will guide you through creating an account and an access token.
* https://docs.predicthq.com/guides/quickstart/

In [ ]:
ACCESS_TOKEN = "z8vSasLdbCFVQlymo4Ng1OPz4GoRLRo3QtpJNRhE"


In [ ]:
DATE_FORMAT = "%Y-%m-%d"
FEATURES_API_URL = "https://api.predicthq.com/v1/features"

phq = Client(access_token=ACCESS_TOKEN)

def get_date_groups(start, end):
    """
    Features API allows a range of up to 90 days, so we have to do several requests
    """

    def _split_dates(s, e):
        capacity = timedelta(days=90)
        interval = 1 + int((e - s) / capacity)
        for i in range(interval):
            yield s + capacity * i
        yield e

    dates = list(_split_dates(start, end))
    for i, (d1, d2) in enumerate(zip(dates, dates[1:])):
        if d2 != dates[-1]:
            d2 -= timedelta(days=1)
        yield d1.strftime(DATE_FORMAT), d2.strftime(DATE_FORMAT)


#### Prepare features for attended and holiday events

In [ ]:
ATTENDED_FEATURES = [
    "phq_attendance_community",
    "phq_attendance_concerts",
    "phq_attendance_conferences",
    "phq_attendance_expos",
    "phq_attendance_festivals",
    "phq_attendance_performing_arts",
    "phq_attendance_sports",
]
HOLIDAY_FEATURES = [
    "phq_rank_observances",
    "phq_rank_public_holidays",
]


def get_features_api_data(lat, lon, start, end, radius=500, rank_threshold=30):
    start = datetime.strptime(start, DATE_FORMAT).date()
    end = datetime.strptime(end, DATE_FORMAT).date()

    result = []
    for gte, lte in get_date_groups(start, end):
        query = {
            "active__gte": gte,
            "active__lte": lte,
            "location__geo": {"lat": lat, "lon": lon, "radius": f"{radius}m"},
        }

        query.update({f"{f}__stats": ["sum"] for f in ATTENDED_FEATURES})
        query.update(
            {f"{f}__phq_rank": {"gte": rank_threshold} for f in ATTENDED_FEATURES}
        )
        query.update({f"{f}": True for f in HOLIDAY_FEATURES})

        features = phq.features.obtain_features(**query)

        for feature in features:
            record = {}
            for k, v in feature.to_dict().items():
                if k == "date":
                    record[k] = v.strftime("%Y-%m-%d")
                elif k in ATTENDED_FEATURES:
                    record[k] = v.get("stats", {}).get("sum")
                elif k in HOLIDAY_FEATURES:
                    record[k] = sum(float(x) for x in v.get("rank_levels", {}).values())

            result.append(record)

    return result


res = get_features_api_data(41.657871, -91.534637, "2021-06-01", "2022-07-04", 1760, 30)
df_attended_holidays = pd.DataFrame(res)

columns_constant = [
    col
    for col in df_attended_holidays.columns[1:]
    if col.replace("phq_attendance_", "").replace("phq_rank_", "")
    not in category_important_results
]
df_attended_holidays.drop(columns=columns_constant, inplace=True)
#df_attended_holidays.head(20)

#### Prepare features for school holidays

In [ ]:
ATTENDED_FEATURES = [
    "phq_attendance_school_holidays",
]


def get_features_api_school_holidays(lat, lon, start, end, rank_threshold=30):
    start = datetime.strptime(start, DATE_FORMAT).date()
    end = datetime.strptime(end, DATE_FORMAT).date()

    result = []
    for gte, lte in get_date_groups(start, end):
        query = {
            "active__gte": gte,
            "active__lte": lte,
            "location__geo": {"lat": lat, "lon": lon, "radius": "1m"},
        }

        query.update({f"{f}__stats": ["sum"] for f in ATTENDED_FEATURES})
        query.update(
            {f"{f}__phq_rank": {"gte": rank_threshold} for f in ATTENDED_FEATURES}
        )

        features = phq.features.obtain_features(**query)

        for feature in features:
            record = {}
            for k, v in feature.to_dict().items():
                if k == "date":
                    record[k] = v.strftime("%Y-%m-%d")
                elif k in ATTENDED_FEATURES:
                    record[k] = v.get("stats", {}).get("sum")

            result.append(record)

    return result


res = get_features_api_school_holidays(
    41.657871, -91.534637, "2021-06-01", "2022-07-04", 30
)
df_school_holidays = pd.DataFrame(res)


#### Prepare features for severe weather events

In [ ]:
SEVERE_WEATHER_FEATURES = {
    "phq_impact_severe_weather_air_quality_retail",
    "phq_impact_severe_weather_blizzard_retail",
    "phq_impact_severe_weather_cold_wave_retail",
    "phq_impact_severe_weather_cold_wave_snow_retail",
    "phq_impact_severe_weather_cold_wave_storm_retail",
    "phq_impact_severe_weather_dust_retail",
    "phq_impact_severe_weather_dust_storm_retail",
    "phq_impact_severe_weather_flood_retail",
    "phq_impact_severe_weather_heat_wave_retail",
    "phq_impact_severe_weather_hurricane_retail",
    "phq_impact_severe_weather_thunderstorm_retail",
    "phq_impact_severe_weather_tornado_retail",
    "phq_impact_severe_weather_tropical_storm_retail",
}


def get_features_api_severe_weather_events(lat, lon, start, end, rank_threshold=30):
    start = datetime.strptime(start, DATE_FORMAT).date()
    end = datetime.strptime(end, DATE_FORMAT).date()

    result = []
    for gte, lte in get_date_groups(start, end):
        query = {
            "active__gte": gte,
            "active__lte": lte,
            "location__geo": {"lat": lat, "lon": lon, "radius": "1m"},
        }

        query.update({f"{f}__stats": ["max"] for f in SEVERE_WEATHER_FEATURES})
        query.update(
            {f"{f}__phq_rank": {"gte": rank_threshold} for f in SEVERE_WEATHER_FEATURES}
        )

        features = phq.features.obtain_features(**query)

        for feature in features:
            record = {}
            for k, v in feature.to_dict().items():
                if k == "date":
                    record[k] = v.strftime("%Y-%m-%d")
                else:
                    record[k] = v.get("stats", {}).get("max")

            result.append(record)

    return result



res = get_features_api_severe_weather_events(
    41.657871, -91.534637, "2021-06-01", "2022-07-04", 60
)
df_severe_weather_features = pd.DataFrame(res)

columns_constant = [
    col
    for col in df_severe_weather_features.sum()[1:].to_dict().keys()
    if df_severe_weather_features[col].sum() == 0
]
df_severe_weather_features.drop(columns=columns_constant, inplace=True)
#df_severe_weather_features.head(20)

In [ ]:
df_event_features = df_attended_holidays.merge(
        df_severe_weather_features, on="date", how="left"
    ).merge(df_school_holidays, on="date", how="left")

df_event_features.tail(5)

,date,phq_attendance_concerts,phq_attendance_expos,phq_attendance_performing_arts,phq_attendance_sports,phq_rank_observances,phq_rank_public_holidays,phq_impact_severe_weather_cold_wave_retail,phq_impact_severe_weather_cold_wave_storm_retail,phq_impact_severe_weather_flood_retail,phq_impact_severe_weather_heat_wave_retail,phq_impact_severe_weather_thunderstorm_retail,phq_impact_severe_weather_tornado_retail,phq_attendance_school_holidays
394,2022-06-30,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15831.0
395,2022-07-01,429.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15831.0
396,2022-07-02,429.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,9.0,0.0,0.0,15831.0
397,2022-07-03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,16.0,0.0,0.0,15831.0
398,2022-07-04,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,37.0,0.0,0.0,15831.0


### Severe weather events with Demand Impact Patterns and Polygons

- Severe weather is one of the most impactful event categories in the world. Severe weather warnings or alerts may lead to disruption and can have a huge influence on demand.

- Severe weather events impact demand before and after the event itself. PredictHQ’s Demand Impact Patterns accurately capture the leading, lagging and coincident effects of a severe weather event on demand. It's built with advanced ML models by using the global severe weather events and our proprietary global demand data lake. 

- Get access to our easy to use severe weather event features for your forecasts through Features API.

![Picture title](data/swdip.png)

#### Load demand via CSV file

In [ ]:
# Load demand dataset
df_demand = pd.read_csv("data/demand.csv")
df_demand["date"] = pd.to_datetime(df_demand["date"])

### Combine event features with time trend features

In [ ]:
# Convert date to time relevant feature
df_event_features["date"] = pd.to_datetime(df_event_features["date"])
df_event_features[["day_of_week", "week_of_year", "month_of_year"]] = (
    df_event_features["date"]
    .map(lambda x: [x.day_of_week, x.weekofyear, x.month])
    .to_list()
)
df = df_demand.merge(df_event_features, how="left", on="date")

### Build a forecast using XGBoost model based on all features

In [ ]:
split_date_test = "2022-06-20"
feature_columns = df.columns[2:]
demand_column = "demand"

X_train = df[df["date"] < split_date_test][feature_columns]
X_test = df[df["date"] >= split_date_test][feature_columns]
y_train = df[df["date"] < split_date_test][demand_column]
y_test = df[df["date"] >= split_date_test][demand_column]

In [ ]:
# len(X_train), len(X_test), len(y_train), len(y_test)
feature_columns

Index(['phq_attendance_concerts', 'phq_attendance_expos',
       'phq_attendance_performing_arts', 'phq_attendance_sports',
       'phq_rank_observances', 'phq_rank_public_holidays',
       'phq_impact_severe_weather_cold_wave_retail',
       'phq_impact_severe_weather_cold_wave_storm_retail',
       'phq_impact_severe_weather_flood_retail',
       'phq_impact_severe_weather_heat_wave_retail',
       'phq_impact_severe_weather_thunderstorm_retail',
       'phq_impact_severe_weather_tornado_retail',
       'phq_attendance_school_holidays', 'day_of_week', 'week_of_year',
       'month_of_year'],
      dtype='object')

In [ ]:
xgb_model = XGBRegressor(
    n_estimators=100,
    learning_rate=0.1,
    max_depth=6,
    random_state=42,
    n_jobs=-1,
)
xgb_model.fit(
    X_train,
    y_train,
    verbose=True,
)
# xgb_model.save_model(f"xgb_demand_forecasting.json") #

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.1, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=100,
             n_jobs=-1, num_parallel_tree=1, predictor='auto', random_state=42,
             reg_alpha=0, reg_lambda=1, ...)

In [ ]:
xgb_model.predict(X_test)

array([ 95.840096, 112.99002 , 110.005295, 115.64472 , 106.83255 ,
        86.38655 ,  80.55523 ,  99.003136, 100.996796,  99.324036,
       102.04387 , 109.46677 ,  90.95501 ,  86.45726 ,  88.18276 ],
      dtype=float32)

### Forecast the next two weeks' demand starting from 2022-06-20

In [ ]:
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=df[df["date"] < split_date_test]["date"],
        y=df[df["date"] < split_date_test][demand_column],
        name="y_training",
        mode="lines+markers",
    )
)

fig.add_trace(
    go.Scatter(
        x=df[df["date"] >= split_date_test]["date"],
        y=xgb_model.predict(X_test),
        name="y_prediction",
        mode="lines+markers",
    )
)

fig.add_trace(
    go.Scatter(
        x=df[df["date"] >= split_date_test]["date"],
        y=df[df["date"] >= split_date_test][demand_column],
        name="y_truth",
        mode="lines+markers",
    )
)

# Display the figure
fig.show()

### Compare forecasts with and without event features

In [ ]:
df_withoutevents = df[
    ["date", "demand", "day_of_week", "week_of_year", "month_of_year"]
]
df_withoutevents.head(2)

,date,demand,day_of_week,week_of_year,month_of_year
0,2021-06-01,96,1,22,6
1,2021-06-02,97,2,22,6


In [ ]:
feature_columns_withoutevents = df_withoutevents.columns[2:]
X_train_withoutevents = df_withoutevents[df_withoutevents["date"] < split_date_test][
    feature_columns_withoutevents
]
X_test_withoutevents = df_withoutevents[df_withoutevents["date"] >= split_date_test][
    feature_columns_withoutevents
]
y_train_withoutevents = df_withoutevents[df_withoutevents["date"] < split_date_test][
    demand_column
]
y_test_withoutevents = df_withoutevents[df_withoutevents["date"] >= split_date_test][
    demand_column
]

In [ ]:
xgb_model_withoutevents = XGBRegressor(
    n_estimators=100,
    learning_rate=0.1,
    max_depth=6,
    random_state=42,
    n_jobs=-1,
)
xgb_model_withoutevents.fit(
    X_train_withoutevents,
    y_train_withoutevents,
)

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.1, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=100,
             n_jobs=-1, num_parallel_tree=1, predictor='auto', random_state=42,
             reg_alpha=0, reg_lambda=1, ...)

In [ ]:
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=df_withoutevents[df_withoutevents["date"] < split_date_test]["date"],
        y=df_withoutevents[df_withoutevents["date"] < split_date_test][demand_column],
        name="y_training",
        mode="lines+markers",
    )
)

fig.add_trace(
    go.Scatter(
        x=df_withoutevents[df_withoutevents["date"] >= split_date_test]["date"],
        y=xgb_model_withoutevents.predict(X_test_withoutevents),
        name="y_prediction_no_events",
        mode="lines+markers",
    )
)

fig.add_trace(
    go.Scatter(
        x=df_withoutevents[df_withoutevents["date"] >= split_date_test]["date"],
        y=df_withoutevents[df_withoutevents["date"] >= split_date_test][demand_column],
        name="y_truth",
        mode="lines+markers",
    )
)
fig.add_trace(
    go.Scatter(
        x=df[df["date"] >= split_date_test]["date"],
        y=xgb_model.predict(X_test),
        name="y_prediction_withevents",
        mode="lines+markers",
    )
)
# Display the figure
fig.show()

### Model comparison based on Mean Absolute Error (MAE) and Root Mean Square Error (RMSE)

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

MAE_model_withevents = mean_absolute_error(y_test, xgb_model.predict(X_test))
MAE_model_no_events = mean_absolute_error(
    y_test_withoutevents, xgb_model_withoutevents.predict(X_test_withoutevents)
)
MAE_Model_improvement = (
    (MAE_model_no_events - MAE_model_withevents) / MAE_model_no_events * 100
)

RMSE_model_withevents = mean_squared_error(
    y_test, xgb_model.predict(X_test), squared=False
)
RMSE_model_no_events = mean_squared_error(
    y_test_withoutevents,
    xgb_model_withoutevents.predict(X_test_withoutevents),
    squared=False,
)
RMSE_Model_improvement = (
    (RMSE_model_no_events - RMSE_model_withevents) / RMSE_model_no_events * 100
)

print(f"With event features in the model, MAE is {MAE_model_withevents:.2f}")
print(f"Without event features in the model, MAE is {MAE_model_no_events:.2f}")
print(
    f"With event features in the model, MAE improved by {MAE_Model_improvement:.2f}%"
)
print(" ")
print(f"With event features in the model, RSME is {RMSE_model_withevents:.2f}")
print(f"Without event features in the model, RSME is {RMSE_model_no_events:.2f}")
print(
    f"With event features in the model, RSME improved by {RMSE_Model_improvement:.2f}%"
)

With event features in the model, MAE is 9.41
Without event features in the model, MAE is 11.23
With event features in the model, MAE improved by 16.21%
 
With event features in the model, RSME is 10.74
Without event features in the model, RSME is 14.04
With event features in the model, RSME improved by 23.49%


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=2da4cf6e-5d8b-462a-83f9-4b3699bcaaa1' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>